#Samer Baslan
#CMPE-297: Assignment 7 Active Learning
#Fall 2021

Resource/credit: https://colab.research.google.com/github/rachhouse/intro-to-data-linking/blob/main/tutorial_notebooks/03_Link_FEBRL_Data_with_Active_Learning.ipynb#scrollTo=1JOAImRfTljQ

In [1]:
try:
    import google.colab
    COLAB = True
except ModuleNotFoundError:
    COLAB = False
    
# If we're running in Colab, download the tutorial functions file 
# to the Colab session local directory, and install required libraries.
if COLAB:
    import requests
    
    tutorial_functions_url = "https://raw.githubusercontent.com/rachhouse/intro-to-data-linking/main/tutorial_notebooks/linking_tutorial_functions.py"
    r = requests.get(tutorial_functions_url)
    
    with open("linking_tutorial_functions.py", "w") as fh:
        fh.write(r.text)
    
    !pip install -q altair dedupe dedupe-variable-name jellyfish recordlinkage 

In [2]:
!pip install numpy --upgrade

In [3]:
import datetime
import itertools
import os
import pathlib
import re
from typing import Any, Dict, Optional

import dedupe
import pandas as pd

import linking_tutorial_functions as tutorial

INFO:root:Generating grammar tables from /usr/lib/python3.7/lib2to3/Grammar.txt
INFO:root:Generating grammar tables from /usr/lib/python3.7/lib2to3/PatternGrammar.txt


In [4]:
WORKING_DIR = pathlib.Path(os.path.abspath(''))
WORKING_DIR

PosixPath('/content')

In [5]:
df_A, df_B, df_ground_truth = tutorial.load_febrl_training_data(COLAB)

In [6]:
df_A.head()

,first_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,age,phone_number,soc_sec_id
person_id_A,,,,,,,,,,,,
fbc4143d-15f9-4f27-b5f0-dedbadce6616,matilda,struck,8,ballard place,,west perth,2470,qld,19611002,32,03 05903135,8276847
48a56cad-7ba6-45e1-97cd-517ba65bdab5,lachlan,eglinton,36,kambalda crescent,villa 427,auburn,5109,,19260108,27,,9937958
b1792d21-e4be-4b86-8dea-454ffa5194c5,mikayla,asher,588,britten-jones drive,,miami,4218,nsw,19251102,32,03 33770501,7017310
96653d73-bebc-4459-94f3-c3f0a8c514d4,grace,bristow,7,,wandella park snowy,cardiff,6163,nsw,19400120,,07 37864073,3535974
41f038b8-77c0-45a5-9e1f-e62b8637ffd1,wilson,bishop,11,chisholm street,,bronte,2490,nsw,19210305,27,04 15209769,5573522


In [7]:
def format_dob(dob: str) -> Optional[str]:
    """ Transform date of birth format from YYYYMMDD to mm/dd/yy.
        If DOB cannot be transformed, return None.
    """
    try:
        if re.match(r"\d{8}", dob):
            return (datetime.datetime.strptime(dob, "%Y%m%d")).strftime("%m/%d/%y")
    except:
        pass

    return None

def strip_and_null(x: Any) -> Optional[str]:
    """ Stringify incoming variable, remove trailing/leading whitespace
        and return resulting string. Return None if resulting string is empty.
    """
    x = str(x).strip()
    
    if x == "":
        return None
    else:
        return x
    
def convert_df_to_dict(df: pd.DataFrame) -> Dict[str, Dict]:
    """ Convert pandas DataFrame to dict keyed by record id.
        Convert all fields to strings or Nones to satisfy dedupe.
        Transform date format of date_of_birth field.
    """    

    for col in df.columns:
        df[col] = df[col].apply(lambda x: strip_and_null(x))

    df["date_of_birth"] = df["date_of_birth"].apply(lambda x: format_dob(x))    

    return df.to_dict("index")

In [8]:
records_A = convert_df_to_dict(df_A)
records_B = convert_df_to_dict(df_B)

In [9]:
[records_A[k] for k in list(records_A.keys())[0:2]]

[{'address_1': 'ballard place',
  'address_2': None,
  'age': '32',
  'date_of_birth': '10/02/61',
  'first_name': 'matilda',
  'phone_number': '03 05903135',
  'postcode': '2470',
  'soc_sec_id': '8276847',
  'state': 'qld',
  'street_number': '8',
  'suburb': 'west perth',
  'surname': 'struck'},
 {'address_1': 'kambalda crescent',
  'address_2': 'villa 427',
  'age': '27',
  'date_of_birth': '01/08/26',
  'first_name': 'lachlan',
  'phone_number': None,
  'postcode': '5109',
  'soc_sec_id': '9937958',
  'state': None,
  'street_number': '36',
  'suburb': 'auburn',
  'surname': 'eglinton'}]

In [10]:
%%time

fields = [
    { "field" : "first_name", "type" : "Name" },
    { "field" : "surname", "type" : "Name" },
    { "field" : "address_1", "type" : "ShortString" },
    { "field" : "address_2", "type" : "ShortString" },
    { "field" : "suburb", "type" : "ShortString" },
    { "field" : "postcode", "type" : "Exact" },
    { "field" : "state", "type" : "Exact" },
    { "field" : "date_of_birth", "type" : "DateTime" },
    { "field" : "soc_sec_id", "type" : "Exact" },
]

linker = dedupe.RecordLink(fields)
linker.prepare_training(records_A, records_B)

INFO:dedupe.canopy_index:Removing stop word re
INFO:dedupe.training:Final predicate set:
INFO:dedupe.training:SimplePredicate: (wholeFieldPredicate, date_of_birth)


CPU times: user 46.2 s, sys: 673 ms, total: 46.9 s
Wall time: 46.4 s


In [11]:
dedupe.console_label(linker)

first_name : william
surname : finlay
address_1 : florentine circuit
address_2 : merimaja
suburb : None
postcode : 2099
state : nsw
date_of_birth : 07/31/88
soc_sec_id : 1431428

first_name : william
surname : findoay
address_1 : florentine circuit
address_2 : merimaja
suburb : None
postcode : 2099
state : nsw
date_of_birth : None
soc_sec_id : 1431428

0/10 positive, 0/10 negative
Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished


y


first_name : matilda
surname : piening
address_1 : lenehan street
address_2 : None
suburb : None
postcode : 4552
state : nsw
date_of_birth : 08/11/52
soc_sec_id : 8766201

first_name : lydia
surname : paine
address_1 : None
address_2 : None
suburb : williamstown
postcode : 2232
state : nsw
date_of_birth : None
soc_sec_id : 4266599

1/10 positive, 0/10 negative
Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished / (p)revious


n


INFO:dedupe.training:Final predicate set:
INFO:dedupe.training:SimplePredicate: (wholeFieldPredicate, soc_sec_id)
first_name : danika
surname : longo
address_1 : None
address_2 : alblas hostel
suburb : avondale heights
postcode : 4558
state : nsw
date_of_birth : 07/01/35
soc_sec_id : 9461847

first_name : chloe
surname : hare
address_1 : la trobme close
address_2 : None
suburb : None
postcode : 3195
state : qld
date_of_birth : None
soc_sec_id : 9880358

1/10 positive, 1/10 negative
Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished / (p)revious


...

first_name : jye
surname : trainor
address_1 : summerland circuit
address_2 : None
suburb : coldstream
postcode : 2615
state : nsw
date_of_birth : 08/28/39
soc_sec_id : 4573801

first_name : jye
surname : trainor
address_1 : summerland circuit
address_2 : None
suburb : coldstream
postcode : 2615
state : nsw
date_of_birth : 09/28/39
soc_sec_id : 4573803

5/10 positive, 9/10 negative
Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished / (p)revious


y


first_name : logan
surname : mac onochie
address_1 : mackellar crescent
address_2 : None
suburb : oatlands
postcode : 4207
state : vic
date_of_birth : 08/13/84
soc_sec_id : 4647965

first_name : logt
surname : mac onochie
address_1 : None
address_2 : None
suburb : oatlands
postcode : 4207
state : vic
date_of_birth : None
soc_sec_id : 4648775

6/10 positive, 9/10 negative
Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished / (p)revious


f


Finished labeling
INFO:dedupe.training:Final predicate set:
INFO:dedupe.training:SimplePredicate: (firstTwoTokensPredicate, address_1)
INFO:dedupe.training:SimplePredicate: (wholeFieldPredicate, date_of_birth)


In [12]:
%%time
linker.train()

INFO:rlr.crossvalidation:using cross validation to find optimum alpha...
INFO:rlr.crossvalidation:optimum alpha: 0.000010, score 0.45942743587055035
INFO:dedupe.training:Final predicate set:
INFO:dedupe.training:(SimplePredicate: (wholeFieldPredicate, suburb), SimplePredicate: (wholeFieldPredicate, postcode))
INFO:dedupe.training:(SimplePredicate: (wholeFieldPredicate, address_2), SimplePredicate: (fingerprint, address_1), PartialIndexTfidfNGramSearchPredicate: (0.4, surname, Surname))


CPU times: user 4.07 s, sys: 477 ms, total: 4.55 s
Wall time: 4.09 s


In [13]:
ACTIVE_LEARNING_DIR = WORKING_DIR / "dedupe_active_learning"
ACTIVE_LEARNING_DIR.mkdir(parents=True, exist_ok=True)

SETTINGS_FILE = ACTIVE_LEARNING_DIR / "dedupe_learned_settings"
TRAINING_FILE = ACTIVE_LEARNING_DIR / "dedupe_training.json"

with open(TRAINING_FILE, "w") as fh:
    linker.write_training(fh)
    
with open(SETTINGS_FILE, "wb") as sf:
    linker.write_settings(sf)

In [14]:
linker.predicates

((SimplePredicate: (wholeFieldPredicate, suburb),
  SimplePredicate: (wholeFieldPredicate, postcode)),
 (SimplePredicate: (wholeFieldPredicate, address_2),
  SimplePredicate: (fingerprint, address_1),
  PartialIndexTfidfNGramSearchPredicate: (0.4, surname, Surname)))

In [15]:
candidate_pairs = [x for x in linker.pairs(records_A, records_B)]
print(f"{len(candidate_pairs):,} candidate pairs generated from blocking.")

1,778 candidate pairs generated from blocking.


In [16]:
candidate_pairs[0]

(('48a56cad-7ba6-45e1-97cd-517ba65bdab5',
  {'address_1': 'kambalda crescent',
   'address_2': 'villa 427',
   'age': '27',
   'date_of_birth': '01/08/26',
   'first_name': 'lachlan',
   'phone_number': None,
   'postcode': '5109',
   'soc_sec_id': '9937958',
   'state': None,
   'street_number': '36',
   'suburb': 'auburn',
   'surname': 'eglinton'}),
 ('c77c2c04-4415-4c4d-b248-18dc28fd63d0',
  {'address_1': 'kambalda crescent',
   'address_2': None,
   'age': None,
   'date_of_birth': '01/08/26',
   'first_name': 'lachlan',
   'phone_number': None,
   'postcode': '5109',
   'soc_sec_id': '9937958',
   'state': None,
   'street_number': '366',
   'suburb': 'auburn',
   'surname': 'eglinton'}))

In [17]:
df_candidate_links = pd.DataFrame(
    [(x[0][0], x[1][0]) for x in candidate_pairs]
).rename(columns={0 : "person_id_A", 1 : "person_id_B"}).set_index(["person_id_A", "person_id_B"])

df_candidate_links.head()

,
person_id_A,person_id_B
48a56cad-7ba6-45e1-97cd-517ba65bdab5,c77c2c04-4415-4c4d-b248-18dc28fd63d0
b4e3efc2-9c8f-4e3e-8b98-9bfa842094f9,e63f19ca-3f5b-4021-ac1e-05fc7495bd48
050a4ce1-8fc9-410d-bae1-65a70a518e34,e36dc4e4-c33c-4021-9dba-ceed3a4956d7
7264bfb0-bbcb-4f68-b9bf-03619237cfb2,8e5d98b8-9611-480e-8c65-b0e56520307b
67f406b1-ddbe-4dff-b725-f6653f8af0a6,97bbf64b-d893-4af2-91d8-86215fe0a4f7


In [18]:
max_candidate_pairs = df_A.shape[0]*df_B.shape[0]

print(f"{max_candidate_pairs:,} total possible pairs.")

# Calculate search space reduction.
search_space_reduction = round(1 - len(candidate_pairs)/max_candidate_pairs, 6)
print(f"\n{len(candidate_pairs):,} pairs after full blocking: {search_space_reduction}% search space reduction.")

# Calculate retained true links percentage.
total_true_links = df_ground_truth.shape[0]
true_links_after_blocking = pd.merge(
    df_ground_truth,
    df_candidate_links,
    left_index=True,
    right_index=True,
    how="inner"
).shape[0]

retained_true_link_percent = round((true_links_after_blocking/total_true_links) * 100, 2)
print(f"{retained_true_link_percent}% true links retained after blocking.")

10,562,500 total possible pairs.

1,778 pairs after full blocking: 0.999832% search space reduction.
58.93% true links retained after blocking.


In [19]:
%%time
linked_records = linker.join(records_A, records_B, threshold=0.0, constraint="one-to-one")

CPU times: user 312 ms, sys: 58.7 ms, total: 371 ms
Wall time: 1.35 s


In [20]:
linked_records[0:3]

[(('ff64e7b6-7a23-45d2-abfd-df84b7dfe02a',
   'e44731a3-6d4d-4d1e-8466-57aef723dcdc'),
  1.0),
 (('fe7dcaf6-3a4d-456d-8ac1-2b81875158bc',
   'eb827aae-7870-4a7a-8dc4-f5e755ae19b6'),
  1.0),
 (('fc8bd23d-afdf-4a67-81d8-31fafe560c76',
   '241eb6d8-c988-4838-96e3-1fff5e778bd4'),
  1.0)]

In [21]:
df_predictions = pd.DataFrame(
    [ {"person_id_A" : x[0][0], "person_id_B" : x[0][1], "model_score" : x[1]} for x in linked_records]
)

df_predictions = df_predictions.set_index(["person_id_A", "person_id_B"])

df_predictions = pd.merge(
    df_predictions,
    df_ground_truth,
    left_index=True,
    right_index=True,
    how="left",
)

df_predictions["ground_truth"].fillna(False, inplace=True)
df_predictions

,,model_score,ground_truth
person_id_A,person_id_B,,
ff64e7b6-7a23-45d2-abfd-df84b7dfe02a,e44731a3-6d4d-4d1e-8466-57aef723dcdc,1.000000,True
fe7dcaf6-3a4d-456d-8ac1-2b81875158bc,eb827aae-7870-4a7a-8dc4-f5e755ae19b6,1.000000,True
fc8bd23d-afdf-4a67-81d8-31fafe560c76,241eb6d8-c988-4838-96e3-1fff5e778bd4,1.000000,True
fc7ce04c-b9de-428b-83c2-80219ad8c4d3,d0e57003-6a0f-4e51-9cff-0e60927e0611,1.000000,True
fc041887-b5b8-427b-ac8b-bd867d0c813f,f93ba54b-0c5c-400d-89e8-1fe6746bf928,1.000000,True
...,...,...,...
357f0731-043e-4fe6-a5e9-cd65cdc063f0,922ae285-bfc7-463a-ac75-dda307b09ac5,0.000057,True
6a85e26a-a969-43d3-b142-d635593f091a,4016047d-6a37-48c2-8549-0aa0b5f8a78a,0.000046,True
e6be8689-4229-44f6-8399-4e58e0b159cd,326d11f5-1ae2-4d68-8ce7-0258b6576c05,0.000045,True


In [22]:
df_predictions["ground_truth"].value_counts()

True     1761
False       7
Name: ground_truth, dtype: int64

In [23]:
tutorial.plot_model_score_distribution(df_predictions)

INFO:numexpr.utils:NumExpr defaulting to 2 threads.


alt.Chart(...)

In [24]:
df_eval = tutorial.evaluate_linking(
    df=df_predictions
)

In [25]:
df_eval.head()

,threshold,tp,fp,tn,fn,precision,recall,f1
0,0.000000,1761,7,0,0,0.996041,1.000000,0.998016
1,0.020408,1732,7,0,29,0.995975,0.983532,0.989714
2,0.040816,1719,7,0,42,0.995944,0.976150,0.985948
3,0.061224,1715,7,0,46,0.995935,0.973878,0.984783
4,0.081633,1712,7,0,49,0.995928,0.972175,0.983908


In [26]:
tutorial.plot_precision_recall_vs_threshold(df_eval)

alt.LayerChart(...)

In [27]:
%%time

fields = [
    { "field" : "first_name", "type" : "Name" },
    { "field" : "surname", "type" : "Name" },
    { "field" : "suburb", "type" : "ShortString" },
    { "field" : "postcode", "type" : "Exact" },
    { "field" : "state", "type" : "Exact" },
    { "field" : "date_of_birth", "type" : "DateTime" },
    { "field" : "soc_sec_id", "type" : "Exact" },
]

linker2 = dedupe.RecordLink(fields)

with open(TRAINING_FILE, "r") as fh:
    linker2.prepare_training(records_A, records_B, training_file=fh)

INFO:dedupe.api:reading training from file
INFO:dedupe.training:Final predicate set:
INFO:dedupe.training:SimplePredicate: (dayPredicate, date_of_birth)
INFO:dedupe.training:Final predicate set:
INFO:dedupe.training:SimplePredicate: (wholeFieldPredicate, soc_sec_id)
INFO:dedupe.training:SimplePredicate: (dayPredicate, date_of_birth)
INFO:dedupe.training:PartialPredicate: (firstTokenPredicate, first_name, Surname)


CPU times: user 35.1 s, sys: 629 ms, total: 35.7 s
Wall time: 35.1 s


In [28]:
dedupe.console_label(linker2)

first_name : alexander
surname : coleman
suburb : erskine
postcode : 3181
state : wa
date_of_birth : 06/27/65
soc_sec_id : 5751582

first_name : stathya
surname : lucy
suburb : erskine
postcode : 5260
state : nsw
date_of_birth : 05/24/74
soc_sec_id : 5726885

6/10 positive, 9/10 negative
Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished


n


first_name : mitchell
surname : plane
suburb : torrumbarry
postcode : 2705
state : nsw
date_of_birth : 01/23/69
soc_sec_id : 5703316

first_name : liama
surname : hiloon
suburb : murrumbateman
postcode : 3178
state : vic
date_of_birth : 01/03/73
soc_sec_id : 6500763

6/10 positive, 10/10 negative
Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished / (p)revious


n


first_name : grace
surname : tenholder
suburb : moura
postcode : 6101
state : sa
date_of_birth : 04/19/27
soc_sec_id : 4104938

first_name : graje
surname : tenholder
suburb : moura
postcode : 6101
state : sa
date_of_birth : 04/29/27
soc_sec_id : 4144938

6/10 positive, 11/10 negative
Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished / (p)revious


y


first_name : eloise
surname : povey
suburb : lismore
postcode : 6170
state : nsw
date_of_birth : 08/09/34
soc_sec_id : 7658579

first_name : eloise
surname : povey
suburb : lismore
postcode : 6170
state : nsw
date_of_birth : 05/09/34
soc_sec_id : 7487027

7/10 positive, 11/10 negative
Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished / (p)revious


f


Finished labeling
INFO:dedupe.training:Final predicate set:
INFO:dedupe.training:SimplePredicate: (wholeFieldPredicate, postcode)


In [29]:
%%time
linker2.train()

INFO:rlr.crossvalidation:using cross validation to find optimum alpha...
/usr/local/lib/python3.7/dist-packages/rlr/crossvalidation.py:122: RuntimeWarning: invalid value encountered in double_scalars
  * (true_distinct + false_distinct)))
INFO:rlr.crossvalidation:optimum alpha: 0.000010, score 0.6007630668398516
INFO:dedupe.training:Final predicate set:
INFO:dedupe.training:(SimplePredicate: (wholeFieldPredicate, suburb), SimplePredicate: (wholeFieldPredicate, postcode))
INFO:dedupe.training:(SimplePredicate: (wholeFieldPredicate, postcode), PartialIndexTfidfNGramSearchPredicate: (0.4, surname, Surname), SimplePredicate: (suffixArray, first_name))


CPU times: user 5.01 s, sys: 841 ms, total: 5.85 s
Wall time: 5 s


In [30]:
candidate_pairs = [x for x in linker2.pairs(records_A, records_B)]
print(f"{len(candidate_pairs):,} candidate pairs generated from blocking.")

df_candidate_links = pd.DataFrame(
    [(x[0][0], x[1][0]) for x in candidate_pairs]
).rename(columns={0 : "person_id_A", 1 : "person_id_B"}).set_index(["person_id_A", "person_id_B"])

max_candidate_pairs = df_A.shape[0]*df_B.shape[0]

print(f"{max_candidate_pairs:,} total possible pairs.")

# Calculate search space reduction.
search_space_reduction = round(1 - len(candidate_pairs)/max_candidate_pairs, 6)
print(f"\n{len(candidate_pairs):,} pairs after full blocking: {search_space_reduction}% search space reduction.")

# Calculate retained true links percentage.
total_true_links = df_ground_truth.shape[0]
true_links_after_blocking = pd.merge(
    df_ground_truth,
    df_candidate_links,
    left_index=True,
    right_index=True,
    how="inner"
).shape[0]

retained_true_link_percent = round((true_links_after_blocking/total_true_links) * 100, 2)
print(f"{retained_true_link_percent}% true links retained after blocking.")

1,895 candidate pairs generated from blocking.
10,562,500 total possible pairs.

1,895 pairs after full blocking: 0.999821% search space reduction.
62.83% true links retained after blocking.


In [31]:
%%time
linked_records = linker2.join(records_A, records_B, threshold=0.0, constraint="one-to-one")

CPU times: user 429 ms, sys: 84.1 ms, total: 513 ms
Wall time: 1.67 s


In [32]:
df_predictions = pd.DataFrame(
    [ {"person_id_A" : x[0][0], "person_id_B" : x[0][1], "model_score" : x[1]} for x in linked_records]
)

df_predictions = df_predictions.set_index(["person_id_A", "person_id_B"])

df_predictions = pd.merge(
    df_predictions,
    df_ground_truth,
    left_index=True,
    right_index=True,
    how="left",
)

df_predictions["ground_truth"].fillna(False, inplace=True)
df_predictions

,,model_score,ground_truth
person_id_A,person_id_B,,
ffd668ac-2f63-4c05-a6a3-58ebcf1f4a80,3e8c4b67-3611-4a08-84c8-b082b627bb21,1.000000,True
ff86e492-166d-4652-bf5b-61b9eef60e51,0e4b371a-3c2b-4e2d-bae3-1e37058855b7,1.000000,True
ff64e7b6-7a23-45d2-abfd-df84b7dfe02a,e44731a3-6d4d-4d1e-8466-57aef723dcdc,1.000000,True
ff35c358-732f-40ed-a686-bd228089323a,79160577-d48f-4434-9ff1-af16a4eebf4c,1.000000,True
fee2a7e5-2a4e-4930-acad-4ecd7233d296,ad491112-9c0f-4364-b892-618538c9f673,1.000000,True
...,...,...,...
3e873282-5d54-497c-8147-7172bfb21b2a,73932629-9b1d-4d4e-9133-471e66d9132e,0.178749,True
49d16fd6-efea-4b61-80c5-800eac83547a,6a016ff5-9b27-417e-9289-0f19548f22e0,0.157257,True
49ce1803-3d4f-4a7a-a438-deed4453cdae,8f1d0229-d893-43a6-a04f-40dfe2c9a497,0.155047,True


In [33]:
df_predictions["ground_truth"].value_counts()

True    1885
Name: ground_truth, dtype: int64

In [34]:
tutorial.plot_model_score_distribution(df_predictions)

alt.Chart(...)

In [35]:
df_eval = tutorial.evaluate_linking(
    df=df_predictions
)

tutorial.plot_precision_recall_vs_threshold(df_eval)

alt.LayerChart(...)